# sqlalchemy note

In [1]:
import sqlalchemy
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy import Column, Integer, String
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
sqlalchemy.__version__

'1.1.4'

## 创建示例数据库

In [2]:
# engine = create_engine('sqlite:///:memory:', echo=True) #显示引擎信息
engine = create_engine('sqlite:///:memory:', echo=False)
Base = declarative_base()


class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(
        String,
        default="未命名",
        info={"width":50, "heigh":100}
    )
    fullname = Column(String)
    password = Column(String)
    
    level_id = Column(Integer, ForeignKey('level.id'), default=1)
    level = relationship("Level")

    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (
            self.name, self.fullname, self.password)
    
    def as_dict(self):
        values = {}
        for col in self.__table__.columns:
            name = col.name
            value = getattr(self, name)
            values[name] = value
        return values
        
        

class Level(Base):
    __tablename__ = 'level'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)

Base.metadata.create_all(engine)

## 添加数据

In [3]:
Session = sessionmaker(bind=engine)
session = Session()

In [4]:
# 多条记录
session.add_all([
        Level(name='admin'),
        Level(name='normal'),
])
session.commit()

In [5]:
# 单条记录
session.add(User(name='wendy', fullname='Wendy Williams', password='foobar', level_id=0))
user_mary = User(name='mary', fullname='Mary Contrary', password='xxg527', level=Level(name='admin'))
session.add(user_mary)
session.commit()

## 读取数据

In [6]:
mary = session.query(User).filter_by(name='mary').first()
print("fullname:", mary.fullname)
print("password:", mary.password)
print("level:", mary.level.name)
print(mary.as_dict())

fullname: Mary Contrary
password: xxg527
level: admin
{'fullname': 'Mary Contrary', 'id': 2, 'password': 'xxg527', 'level_id': 3, 'name': 'mary'}


In [7]:
dir(mary)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__mapper__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__table__',
 '__tablename__',
 '__weakref__',
 '_decl_class_registry',
 '_sa_class_manager',
 '_sa_instance_state',
 'as_dict',
 'fullname',
 'id',
 'level',
 'level_id',
 'metadata',
 'name',
 'password']

In [8]:
mary.__table__
dir(mary.__table__)

['__and__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__invert__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__visit_name__',
 '__weakref__',
 '_annotate',
 '_annotations',
 '_autoincrement_column',
 '_autoload',
 '_clone',
 '_cloned_set',
 '_cols_populated',
 '_columns',
 '_compiler',
 '_compiler_dispatch',
 '_constructor',
 '_copy_internals',
 '_deannotate',
 '_execute_on_connection',
 '_extra_dependencies',
 '_extra_kwargs',
 '_from_objects',
 '_hide_froms',
 '_init',
 '_init_collections',
 '_init_existing',
 '_init_items',
 '_is_clone_of',
 '_is_from_container',
 '_is_join',
 '_is_lexical_equivalent',
 '_is_select',
 '_kw_reg_for_dialect',
 '_kw_reg_for_dialect_cls',
 '_kw_regis

In [9]:
mary.__table__.foreign_keys

{ForeignKey('level.id')}

## 更新数据

In [10]:
mary.fullname = "玛丽康利"
session.commit()
mary = session.query(User).filter_by(name='mary').first()
print("fullname:", mary.fullname, "||password:", mary.password)

fullname: 玛丽康利 ||password: xxg527


In [12]:
newdata = {"fullname": "mary.tom"}
mary.__dict__.update(newdata)
print("fullname:", mary.fullname, "||password:", mary.password)

fullname: mary.tom ||password: xxg527


In [ ]:
table = User.__table__
print(dict(table.columns))

In [ ]:
table = User.__table__
cols = table.columns
for col in cols:
    print(col.name, col.type, type(col.type), isinstance(col.type, String), col.info)

In [ ]:
dir(Column)

关于设置字段缺省值，参见：http://docs.sqlalchemy.org/en/latest/core/defaults.html